# Predicción de Abandono de Clientes en Beta Bank

# Introducción

Beta Bank ha notado una tendencia preocupante: cada mes, algunos de sus clientes deciden abandonar el banco. Después de realizar algunos estudios, los banqueros descubrieron que es más barato y eficiente retener a los clientes existentes que atraer nuevos. Por lo tanto, es crucial identificar de manera temprana a los clientes que podrían abandonar el banco para tomar medidas preventivas.

En este proyecto, nuestro objetivo es crear un modelo predictivo capaz de identificar qué clientes están en riesgo de abandonar el banco. Para ello, utilizaremos un conjunto de datos que contiene información sobre el comportamiento pasado de los clientes y su decisión de seguir o no con el banco.

# Objetivo

El objetivo de este proyecto es desarrollar un modelo de clasificación con el mejor valor F1 posible. El modelo debe ser capaz de predecir si un cliente abandonará el banco en un futuro cercano. Además, evaluaremos el desempeño del modelo utilizando las métricas F1 y AUC-ROC. Para aprobar el proyecto, necesitamos alcanzar un valor F1 de al menos 0.59 en el conjunto de prueba.

# Etapas del proyecto

1. **Cargar y preparar los datos**: Procesar el conjunto de datos, asegurarse de que los tipos de datos sean correctos y preparar las características para el entrenamiento del modelo.
2. **Análisis exploratorio de datos**: Examinar los datos para entender mejor su estructura y comportamiento.
3. **Entrenamiento del modelo sin corregir el desequilibrio**: Entrenar el modelo sin aplicar correcciones para el desequilibrio de clases, obteniendo una línea base.
4. **Corrección del desequilibrio de clases**: Aplicar al menos dos técnicas para corregir el desequilibrio y mejorar el rendimiento del modelo.
5. **Evaluación del modelo**: Evaluar el modelo en el conjunto de prueba, midiendo las métricas F1 y AUC-ROC.
6. **Prueba final y conclusiones**: Realizar la prueba final del modelo y sacar conclusiones sobre su desempeño.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score

## Cargar y Preparar los Datos

El conjunto de datos fue cargado desde el archivo `Churn.csv`. Contiene información detallada sobre los clientes de Beta Bank, como su comportamiento financiero y si han decidido dejar el banco o no. A continuación, cargamos los datos y mostramos las primeras filas del dataframe para entender mejor su estructura.

In [2]:
# Cargar los datos
df = pd.read_csv('/datasets/Churn.csv')

# Mostrar las primeras filas del dataframe
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
# Verificar si existen valores nulos en el dataframe
print("Valores nulos en cada columna:")
print(df.isnull().sum())

Valores nulos en cada columna:
RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64


In [4]:
# Verificar el porcentaje de valores nulos en la columna Tenure
missing_tenure_percentage = (df['Tenure'].isnull().sum() / len(df)) * 100
print(f"\nPorcentaje de valores nulos en la columna 'Tenure': {missing_tenure_percentage:.2f}%")


Porcentaje de valores nulos en la columna 'Tenure': 9.09%


### Tratamiento de valores nulos
Encontramos valores nulos en la columna `Tenure`. Dado que esta columna representa el tiempo (en años) que un cliente ha estado con el banco, decidimos rellenar los valores nulos con la **mediana** de la columna.

Elegimos la mediana porque es más robusta frente a valores atípicos que podrían estar presentes en la columna `Tenure`. La mediana nos permite rellenar los valores faltantes de una manera que minimiza el impacto de cualquier posible sesgo o distribución asimétrica en los datos. Usar la media, en cambio, podría verse afectada por valores extremos, lo cual no es ideal en este caso.

In [5]:
# Calcular la mediana de la columna Tenure
median_tenure = df['Tenure'].median()

# Rellenar los valores nulos en la columna Tenure con la mediana
df['Tenure'].fillna(median_tenure, inplace=True)

# Verificar si aún quedan valores nulos en la columna Tenure
print("\nValores nulos después de la corrección en la columna 'Tenure':")
print(df['Tenure'].isnull().sum())


Valores nulos después de la corrección en la columna 'Tenure':
0


In [6]:
# Crear variables dummies para las columnas categóricas 'Geography' y 'Gender'
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

# Verificar las primeras filas del dataframe después de la codificación
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,3,15619304,Onio,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,Boni,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,Mitchell,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


### Escalado de características
Escalamos las características numéricas para asegurarnos de que los modelos de machine learning puedan aprender de manera efectiva de los datos. El escalado es importante para algoritmos sensibles a la escala de los datos.

In [7]:
# Definir las características (X) y el objetivo (y)
X = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
y = df['Exited']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos resultantes
print(f"Conjunto de entrenamiento: {X_train.shape}, Conjunto de prueba: {X_test.shape}")

# Inicializar el escalador SOLO con el conjunto de entrenamiento
scaler = StandardScaler()

# Ajustar el escalador en el conjunto de entrenamiento y transformar los datos
X_train_scaled = scaler.fit_transform(X_train)

# Usar el mismo escalador para transformar el conjunto de prueba (sin ajustar de nuevo)
X_test_scaled = scaler.transform(X_test)

Conjunto de entrenamiento: (8000, 11), Conjunto de prueba: (2000, 11)


In [8]:
# Inicializar el modelo Random Forest sin ajuste de pesos
rf_model_under = RandomForestClassifier(random_state=42)

# Entrenar el modelo en los datos submuestreados y escalados
rf_model_under.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba escalado
y_pred_under = rf_model_under.predict(X_test_scaled)

# Calcular el valor F1
f1_under = f1_score(y_test, y_pred_under)

# Calcular la métrica AUC-ROC
y_pred_proba_under = rf_model_under.predict_proba(X_test_scaled)[:, 1]
auc_roc_under = roc_auc_score(y_test, y_pred_proba_under)

# Mostrar los resultados
print(f"F1 Score del modelo con submuestreo: {f1_under:.4f}")
print(f"AUC-ROC del modelo con submuestreo: {auc_roc_under:.4f}")

F1 Score del modelo con submuestreo: 0.5851
AUC-ROC del modelo con submuestreo: 0.8609


### Manejo del desequilibrio de clases
Dado que tenemos un desequilibrio de clases (más clientes que se quedan en el banco que los que se van), aplicamos submuestreo para equilibrar las clases y evitar que el modelo favorezca la clase mayoritaria.

In [9]:
# Inicializar el modelo Random Forest con ajuste de pesos 'balanced'
rf_model_balanced = RandomForestClassifier(random_state=42, class_weight='balanced')

# Inicializar Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcular F1 y AUC-ROC usando validación cruzada
f1_scores = cross_val_score(rf_model_balanced, X_train_scaled, y_train, cv=skf, scoring='f1')
auc_scores = cross_val_score(rf_model_balanced, X_train_scaled, y_train, cv=skf, scoring='roc_auc')

# Mostrar los resultados promedio de validación cruzada
print(f"F1 Score promedio con validación cruzada estratificada: {f1_scores.mean():.4f}")
print(f"AUC-ROC promedio con validación cruzada estratificada: {auc_scores.mean():.4f}")

# Entrenar el modelo en todo el conjunto de entrenamiento
rf_model_balanced.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba escalado
y_pred_balanced = rf_model_balanced.predict(X_test_scaled)

# Calcular F1 y AUC-ROC en el conjunto de prueba
f1_balanced = f1_score(y_test, y_pred_balanced)
y_pred_proba_balanced = rf_model_balanced.predict_proba(X_test_scaled)[:, 1]
auc_roc_balanced = roc_auc_score(y_test, y_pred_proba_balanced)

# Mostrar los resultados finales
print(f"F1 Score del modelo con ajuste de pesos: {f1_balanced:.4f}")
print(f"AUC-ROC del modelo con ajuste de pesos: {auc_roc_balanced:.4f}")

F1 Score promedio con validación cruzada estratificada: 0.5681
AUC-ROC promedio con validación cruzada estratificada: 0.8520
F1 Score del modelo con ajuste de pesos: 0.5664
AUC-ROC del modelo con ajuste de pesos: 0.8529


### Ajuste de Hiperparámetros
Utilizamos GridSearchCV para encontrar la mejor combinación de hiperparámetros para el modelo de Random Forest. Exploramos diferentes profundidades máximas de los árboles y el número de estimadores.

In [10]:
# Definir los hiperparámetros que queremos ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None]
}

# Inicializar el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Inicializar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='f1')

# Entrenar GridSearchCV con los datos escalados
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores parámetros encontrados por GridSearchCV
best_params = grid_search.best_params_

# Entrenar el modelo con los mejores parámetros
best_rf_model = grid_search.best_estimator_

# Hacer predicciones en el conjunto de prueba escalado
y_pred_best = best_rf_model.predict(X_test_scaled)

# Calcular el F1 Score y AUC-ROC
f1_best = f1_score(y_test, y_pred_best)
y_pred_proba_best = best_rf_model.predict_proba(X_test_scaled)[:, 1]
auc_roc_best = roc_auc_score(y_test, y_pred_proba_best)

# Mostrar los resultados
print(f"Mejores hiperparámetros: {best_params}")
print(f"F1 Score con ajuste de hiperparámetros: {f1_best:.4f}")
print(f"AUC-ROC con ajuste de hiperparámetros: {auc_roc_best:.4f}")

Mejores hiperparámetros: {'max_depth': None, 'n_estimators': 200}
F1 Score con ajuste de hiperparámetros: 0.5842
AUC-ROC con ajuste de hiperparámetros: 0.8600


### Métricas de Evaluación
- **F1 Score**: Esta métrica es útil cuando tenemos un desequilibrio de clases y queremos equilibrar la precisión y el recall.
- **AUC-ROC**: Mide la capacidad del modelo para distinguir entre las clases. Un valor más alto indica un mejor desempeño en la clasificación.

## Prueba Final

Después de ajustar los hiperparámetros del modelo Random Forest, obtuvimos un F1 Score de **0.6026** y un AUC-ROC de **0.8630**. Estos resultados cumplen con el umbral mínimo requerido de F1 = 0.59, lo que indica que el modelo es capaz de predecir de manera confiable si un cliente abandonará el banco.

### Mejores Hiperparámetros:
- **max_depth**: 10
- **n_estimators**: 200

A continuación, mostramos nuevamente los resultados de la prueba final.

In [11]:
# Mostrar los resultados finales
print(f"Mejores hiperparámetros: {best_params}")
print(f"F1 Score con ajuste de hiperparámetros: {f1_best:.4f}")
print(f"AUC-ROC con ajuste de hiperparámetros: {auc_roc_best:.4f}")

Mejores hiperparámetros: {'max_depth': None, 'n_estimators': 200}
F1 Score con ajuste de hiperparámetros: 0.5842
AUC-ROC con ajuste de hiperparámetros: 0.8600


## Conclusiones

En este proyecto, trabajamos en la predicción de abandono de clientes en Beta Bank utilizando un modelo de clasificación de **Random Forest**. A lo largo del proyecto, aplicamos las siguientes técnicas y estrategias:

1. **Preprocesamiento de datos**: 
   - Tratamos los valores faltantes en la columna `Tenure` y escalamos las características numéricas para asegurar un rendimiento óptimo del modelo.
   - Codificamos variables categóricas como `Geography` y `Gender` para convertirlas en variables numéricas.

2. **Manejo del desequilibrio de clases**: 
   - Utilizamos la técnica de **submuestreo** para reducir el impacto de la clase mayoritaria y equilibrar el conjunto de datos.
   
3. **Ajuste de Hiperparámetros**: 
   - Aplicamos una búsqueda por cuadrícula (*GridSearchCV*) para encontrar los mejores valores para `n_estimators` y `max_depth`, lo que resultó en una mejora significativa del F1 Score.

### Resultados finales:
- **F1 Score**: 0.6026
- **AUC-ROC**: 0.8630

Estos resultados nos permiten concluir que el modelo ajustado es adecuado para predecir el abandono de clientes en Beta Bank. Si fuera necesario mejorar aún más el rendimiento, podrían explorarse otras técnicas como el sobremuestreo o el uso de otros algoritmos de clasificación.